In [1]:
#imports
import pandas as pd
import numpy as np
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import uuid

In [2]:
#Parse Input
raw_text = ""
pdf_reader = PdfReader("Input/input.pdf")
for page in pdf_reader.pages:
    raw_text += page.extract_text()

#Split data
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=150,
    length_function=len,
    is_separator_regex=False,
)
chunks = splitter.split_text(raw_text)
print(len(chunks))


20


In [3]:
#Build input Dataframe
rows = []
for chunk in chunks:
    row = {
        "text": chunk,
        "chunk_id": uuid.uuid4().hex,
    }
    rows = rows + [row]
df = pd.DataFrame(rows)
print(df)

                                                 text  \
0   Review began\n 05/30/2023 \nReview ended\n 06/...   
1   and the burgeoning private health sector syste...   
2   materials, extracting the data, and analyzing ...   
3   1\n2\n \nOpen Access Review\nArticle\n \nDOI:\...   
4   the evaluation of ASHAs from a health systems ...   
5   violence by communities against doctors \n[18,...   
6   care \n[22]\n. During the eighties and the nin...   
7   [33]\n.\nNational Medical Council\nIndia revam...   
8   Improve Efficiency\n \nThe public health syste...   
9   would receive care at little-to-no-cost. This ...   
10  telemedicine for consultations with healthcare...   
11  and monitoring of care for the elderly. It can...   
12  [53]\n.\nIncreasing violence against healthcar...   
13  is able to provide affordable, accessible, qua...   
14  era: time for a revolution\n. Lancet Glob Heal...   
15  Which doctor for primary health care?\nQuality...   
16  Pradhan Mantri Jan Arogaya 

In [4]:
concepts = []

In [6]:
#RUNNING THIS COSTS MONEY!!!!!!!!!!!!!!!
#----------------------------------------------------------
#Get the concepts
from graph_helper import graph_helper

generate_dataframes = True
helper = graph_helper()
if(generate_dataframes):
    concepts = df.apply(lambda row: helper.get_concepts(row[0]))
    print(concepts)
    concepts = concepts.dropna()
    concepts = concepts.reset_index(drop=True)
    concept_list = np.concatenate(concepts).ravel().tolist()
    print(concept_list)
    dfg1 = helper.formatDF(concept_list)

    dfg1.to_csv("Output/graph.csv", sep="|", index=False)
    df.to_csv("Output/chunks.csv", sep="|", index=False)
dfg1.replace("", np.nan, inplace=True)
dfg1.dropna(subset=["node_1", "node_2", 'edge'], inplace=True)
dfg1['count'] = 4 
## Increasing the weight of the relation to 4. 
## We will assign the weight of 1 when later the contextual proximity will be calculated.  
print(dfg1.shape)
print(dfg1)

text        [{'node_1': 'India', 'node_2': 'health indicat...
chunk_id    [{'node_1': 'network graph maker', 'node_2': '...
dtype: object
[{'node_1': 'India', 'node_2': 'health indicators', 'edge': "India's health indicators have improved in recent times but continue to lag behind those of its peer nations."}, {'node_1': 'India', 'node_2': 'population', 'edge': 'The country with a population of 1.3 billion'}, {'node_1': 'India', 'node_2': 'health workers density', 'edge': 'has an estimated active health workers density of doctors and nurses/midwives of 5.0 and 6.0 respectively, for 10,000 persons'}, {'node_1': 'India', 'node_2': 'WHO threshold', 'edge': 'which is much lower than the WHO threshold of 44.5 doctors, nurses, and midwives per 10,000 population'}, {'node_1': 'India', 'node_2': 'skewed inter-state, urban-rural, and public-private sector divide', 'edge': 'The issue is compounded by the skewed inter-state, urban-rural, and public-private sector divide.'}, {'node_1': 'India', 'n

TypeError: formatDF() takes 1 positional argument but 2 were given